<a href="https://colab.research.google.com/github/gevargas/bigdata-management/blob/master/Intro_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baby stepts into **Spark**

## Colab configuration

In [ ]:
SPARK_VERSION   = "3.0.0"
HADOOP_VERSION  = "2.7"
OPENJDK_VERSION = "11"

In [ ]:
import os
os.environ["JAVA_HOME"]  = "/usr/lib/jvm/java-{0}-openjdk-amd64".format(OPENJDK_VERSION)
os.environ["SPARK_HOME"] = "/usr/local/spark"

In [ ]:
# Java
!apt-get update && apt-get install --no-install-recommends -y openjdk-"$OPENJDK_VERSION"-jdk-headless -qq > /dev/null

# Spark
!curl -L --silent https://archive.apache.org/dist/spark/spark-"$SPARK_VERSION"/spark-"$SPARK_VERSION"-bin-hadoop"$HADOOP_VERSION".tgz > spark-"$SPARK_VERSION"-bin-hadoop"$HADOOP_VERSION".tgz
!tar xzf spark-"$SPARK_VERSION"-bin-hadoop"$HADOOP_VERSION".tgz
!rm spark-"$SPARK_VERSION"-bin-hadoop"$HADOOP_VERSION".tgz
!mv spark-"$SPARK_VERSION"-bin-hadoop"$HADOOP_VERSION" /usr/local/spark

!pip install -q findspark

## Pyspark configuration

Modify according to your needs

In [ ]:
import os
import findspark

SPARK_DRIVER_MEMORY   = "8g"

os.environ["PYSPARK_SUBMIT_ARGS"] = "--driver-memory {0} pyspark-shell".format(SPARK_DRIVER_MEMORY)

findspark.init()

# Initializing Spark

* SparkContext creation

In [ ]:
import pyspark

sc = pyspark.SparkContext.getOrCreate()
sc

# Pyspark example

* Read local/HDFS file

In [ ]:
textFile = sc.textFile("sample_data/README.md")

* Count the number of rows in the file

In [ ]:
textFile.count()

* Print first line in file

In [ ]:
textFile.first()

* Count the number of lines containing the word `"dataset"`

In [ ]:
lines = textFile.filter(lambda line: "dataset" in line)
lines.count()

* Collect (i.e. extract from the spark cluster) the lines containing the word `"dataset"`

In [ ]:
lines.collect()

# First steps working with RDD's

Create a parallelized collection holding the numbers 1 to 5:

In [ ]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)

Once created, the distributed dataset (distData) can be operated on in parallel.

For example, we can call distData.reduce(lambda a, b: a + b) to add up the elements of the list.

In [ ]:
distData.reduce(lambda a, b: a + b)

An important parameter for parallel collections is the number of partitions to cut the dataset into.
Spark will run one task for each partition of the cluster.

*   Spark will run one task for each partition of the cluster.
*   Typically you want 2-4 partitions for each CPU in your cluster.

Normally, Spark tries to set the number of partitions automatically based on your cluster. However, you can also set it manually by passing it as a second parameter to parallelize (e.g. sc.parallelize(data, 10)).

Note: some places in the code use the term slices (a synonym for partitions) to maintain backward compatibility.

## RDD operations

In [ ]:
lines = sc.textFile("sample_data/README.md")
lineLengths = lines.map(lambda s: len(s))
totalLength = lineLengths.reduce(lambda a, b: a + b)

In [ ]:
totalLength

The first line defines a base RDD from an external file.
This dataset is not loaded in memory or otherwise acted on: lines is merely a pointer to the file.

The second line defines lineLengths as the result of a map transformation. Again, lineLengths is not immediately computed, due to laziness.

Finally, we run reduce, which is an action.

At this point Spark breaks the computation into tasks to run on separate machines, and each machine runs both its part of the map and a local reduction, returning only its answer to the driver program.

If we also wanted to use lineLengths again later, we could add: